In [140]:
import numpy as np;
import pandas as pd
import plotly.express as px;
import json
import cufflinks as cf

## Initialize Parameters

In [141]:
# Global parameters
NO_OF_DAYS = 365
modelParameter = {}
modelParameter["pi"] = 0.02
modelParameter["tE"] = 3
modelParameter["tI"] = 6
modelParameter["cw"] = 10
modelParameter["ch"] = 5

modelParameter["eBC"] = 2/3  
modelParameter["eLD"] = 1/3
modelParameter["eDB"] = 1/3  
modelParameter["eRZ"] = 1/3
modelParameter["eOZ"] = 1/2
modelParameter["eHS"] = 1/4

modelParameter["aR"] = 0.1
modelParameter["dhsThreshold"] = 10
modelParameter["dzOZThreshold"] = 1
modelParameter["dzRZThreshold"] = 10

modelParameter["dhsOffsetDays"] = 7
modelParameter["dzOZOffsetDays"] = 21
modelParameter["dzRZOffsetDays"] = 7

district = "all"

print(modelParameter)


{'pi': 0.02, 'tE': 3, 'tI': 6, 'cw': 10, 'ch': 5, 'eBC': 0.6666666666666666, 'eLD': 0.3333333333333333, 'eDB': 0.3333333333333333, 'eRZ': 0.3333333333333333, 'eOZ': 0.5, 'eHS': 0.25, 'aR': 0.1, 'dhsThreshold': 10, 'dzOZThreshold': 1, 'dzRZThreshold': 10, 'dhsOffsetDays': 7, 'dzOZOffsetDays': 21, 'dzRZOffsetDays': 7}


## Load and Initialize Data

In [143]:
## Population data

def preload():
    popDataDF = pd.read_csv("../data/" + district + "_population_data.csv")

    ## cMatrix
    cSparseMatrix = pd.read_csv("../output/" + district + "_cmatrix_results.csv")
    
    return popDataDF, cSparseMatrix



In [144]:
## cijMatrix

## Sort distance data by name for easy matrix transformation

def initialise(popDataDF, cSparseMatrix, modelParameter):

    initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")
    r = len(initDataDF.index)
    C_TPart_Matrix = cSparseMatrix['Cij'].to_numpy().reshape(r,r)
   
    initDataDF.sort_values(by=['name'], inplace=True)
    initDataDF.reset_index(drop=True, inplace=True)

    popDataDF.sort_values(by=['name'], inplace=True)
    popDataDF.reset_index(drop=True, inplace=True)


    initDataDF['N'] = popDataDF['N']
    initDataDF['type'] = popDataDF['type']
    initDataDF['district'] = initDataDF['name'].str.split("__",n = 2, expand = True)[1] 

    districts = initDataDF['district'].unique()

    ## Workplace matrix
    Wn = modelParameter["cw"]*C_TPart_Matrix
    Wn
    Wn.sum(axis=1)[0:7]
    
    return initDataDF, popDataDF, C_TPart_Matrix, districts, Wn


In [145]:
popDataDF, cSparseMatrix = preload()

initDataDF, popDataDF, C_TPart_Matrix, districts, Wn = initialise(popDataDF, cSparseMatrix, modelParameter)


## User parameters

In [146]:
userParameterNone = '{ "breakTheChains" : [], "completeLockDowns" :  [], "districtLockDowns" :  [], "hotSpots": [], "redZones": [], "orangeZones": [] }'
userParameterBC = '{ "breakTheChains" : [{"startDay": 0, "endDay": 10000}], "completeLockDowns" :  [], "districtLockDowns" :  [], "hotSpots": [], "redZones": [], "orangeZones": [] }'
userParameterDB = '{ "breakTheChains" : [], "completeLockDowns" :  [], "districtLockDowns" :  [{"startDay": 0, "endDay": 10000}], "hotSpots": [], "redZones": [], "orangeZones": [] }'
userParameterBCDB = '{ "breakTheChains" : [{"startDay": 0, "endDay": 10000}], "completeLockDowns" :  [], "districtLockDowns" :  [{"startDay": 0, "endDay": 10000}], "hotSpots": [], "redZones": [], "orangeZones": [] }'
userParameterDHS = '{"dynamicHotSpots":[{"startDay":1, "endDay":365}]}'
userParameterDZS = '{"dynamicZones":[{"startDay":1, "endDay":365}]}'
userParameterDHSZS = '{"dynamicZones":[{"startDay":1, "endDay":365}],"dynamicHotSpots":[{"startDay":1, "endDay":365}]}'
userParameterCombo = '{"districtLockDowns" :[{"startDay": 0, "endDay": 100}], "dynamicZones":[{"startDay":1, "endDay":365}], "dynamicHotSpots":[{"startDay":1, "endDay":365}]}'


userParameterData = json.loads(userParameterDZS)
userParameterData



{'dynamicZones': [{'startDay': 1, 'endDay': 365}]}

### Testing init Data

In [147]:
## Test Case 1. Monsoon wedding - Tvm
#seedLocation = "Thiruvananthapuram(C)__Thiruvananthapuram"
#seedLocation = "Thiruvananthapuram(C)__Thiruvananthapuram"
seedLocation = "Palakkad(M)__Palakkad"
initDataDF.loc[initDataDF['name']==seedLocation,'I'] = 20
initDataDF[initDataDF['name']==seedLocation]

,name,S,E,I,R,N,type,district
665,Palakkad(M)__Palakkad,130955,0,20,0,130955,M,Palakkad


## Mitigation Strategy Initializations

### Functions

In [148]:
def isMitigationEnabled(day, strategyName, userParameterData):
    if strategyName in userParameterData:
        for strategy in userParameterData[strategyName]:
            if day >= strategy['startDay'] and day <= strategy['endDay']:
                return True
    return False

In [149]:
def get_mitigation_bases(modelParameter):

    #Normal Day
    Mn = np.ones((r,r))

    #Break the chain
    Mbc = Mn*modelParameter["eBC"]
    
    #complete lockdown
    Mld = Mn*modelParameter["eLD"]
    
    return Mn, Mbc, Mld

In [150]:
Mn, Mbc, Mld = get_mitigation_bases(modelParameter)

### 4. District Border Closure

In [151]:
def get_distric_border_closure_mitigation(cSparseMatrix, r, modelParameter):
    cSparseMatrix['iNameDistrict'] = cSparseMatrix['iName'].str.split("__",n = 2, expand = True)[1] 
    cSparseMatrix['jNameDistrict'] = cSparseMatrix['jName'].str.split("__", n = 2, expand = True)[1] 

    cSparseMatrix.loc[cSparseMatrix['iNameDistrict']==cSparseMatrix['jNameDistrict'],'interDistrictFlag'] = 0
    cSparseMatrix.loc[cSparseMatrix['iNameDistrict']!=cSparseMatrix['jNameDistrict'],'interDistrictFlag'] = 1

    Mdb = cSparseMatrix['interDistrictFlag'].to_numpy().reshape(r,r)*modelParameter["eDB"]
    Mdb[Mdb==0] = 1
    return Mdb, cSparseMatrix

In [152]:
Mdb, cSparseMatrix = get_distric_border_closure_mitigation(cSparseMatrix, r, modelParameter)

### 5. Hot spots

In [153]:
def fillRegion(M, i, e):
    M[:,i] = e
    M[i,:] = e
    return M

In [154]:
def getHotSpotMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter):
    M = np.ones((r,r))
    if "hotSpots" not in userParameterData:
        return M
    for hotSpot in userParameterData['hotSpots']:
        if day >= hotSpot['startDay'] and day <= hotSpot['endDay']:
            region = hotSpot['name']
            regionIndex = initDataDF.index[initDataDF['name'] == region][0]
            M = fillRegion(M, regionIndex, modelParameter["eHS"])
    return M


### 6.Red and Orange Zones

In [155]:
def getRedZoneMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter):
    M = np.ones((r,r))
    if 'redZones' not in userParameterData:
        return M
    for redzones in userParameterData['redZones']:
        if day >= redzones['startDay'] and day <= redzones['endDay']:
            district = redzones['district']
            regionsInDistrictList = initDataDF.index[initDataDF['district'] == district]
            for regionsInDistrictIndex in regionsInDistrictList:
                fillRegion(M, regionsInDistrictIndex, modelParameter["eRZ"])
                
            
    return M


In [156]:
def getOrangeZoneMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter):
    M = np.ones((r,r))
    if 'orangeZones' not in userParameterData:
        return M
    for orangezones in userParameterData['orangeZones']:
        if day >= orangezones['startDay'] and day <= orangezones['endDay']:
            district = orangezones['district']
            regionsInDistrictList = initDataDF.index[initDataDF['district'] == district]
            for regionsInDistrictIndex in regionsInDistrictList:
                fillRegion(M, regionsInDistrictIndex, modelParameter["eOZ"])
                
            
    return M


### 7. Dynamic Hot Spots

In [157]:
def updateDynamicHotSpots(day, I, dynamicHotSpots, modelParameter):
    regionIndices = np.where(I*modelParameter["aR"] > modelParameter["dhsThreshold"])[0]
    newDynamicHotSpots = []
    for regionIndex in regionIndices:
        regionIndexUpdated = False;
        for dynamicHotSpot in dynamicHotSpots:
            if regionIndex == dynamicHotSpot["regionIndex"] and dynamicHotSpot["startDay"] <= day < dynamicHotSpot["endDay"]:
                dynamicHotSpot["endDay"] = max(dynamicHotSpot["endDay"], day+modelParameter["dhsOffsetDays"])
                regionIndexUpdated = True            
        
        if not regionIndexUpdated:
            newDynamicHotSpots.append({"regionIndex" : regionIndex, "startDay": day, "endDay":  day+modelParameter["dhsOffsetDays"]})    
    
    return dynamicHotSpots + newDynamicHotSpots

    

In [158]:
def getDynamicHotSpotMitigationMatrix(day, dynamicHotSpots, modelParameter):
    M = np.ones((r,r))
    for dynamicHotSpot in dynamicHotSpots:
        if day >= dynamicHotSpot['startDay'] and day <= dynamicHotSpot['endDay']:
            M = fillRegion(M, dynamicHotSpot['regionIndex'], modelParameter["eHS"])
    return M


### 8. Dynamic Zones

In [159]:
def updateDynamicZones(day, I, districts, dynamicZones, initDataDF, modelParameter):
    #print('-------   DAY-',day,'  ---------- ')
    if "dynamicOrangeZones" not in dynamicZones:
        dynamicZones["dynamicOrangeZones"] = []
    if "dynamicRedZones" not in dynamicZones:
        dynamicZones["dynamicRedZones"] = []
     

    dynamicOrangeZones = dynamicZones["dynamicOrangeZones"];
    dynamicRedZones = dynamicZones["dynamicRedZones"];
    
    newDynamicOrangeZones = []
    newDynamicRedZones = []
    

    for district in districts:
        districtISum = I[initDataDF.index[initDataDF["district"] == district]].sum()
        #if districtISum > 0:
        #    print('District ', district, ' ', districtISum, ' cases')
        
        ## Orange Zone
        if modelParameter["dzOZThreshold"] <= districtISum*modelParameter["aR"] < modelParameter["dzRZThreshold"]:
            #print(district, ' in Orange Zone')
            dynamicOrangeZoneUpdated = False;
            for dynamicOrangeZone in dynamicOrangeZones:
                if district == dynamicOrangeZone["district"]:
                    dynamicOrangeZone["endDay"] =  max(dynamicOrangeZone["endDay"], day+modelParameter["dzOZOffsetDays"])
                    dynamicOrangeZoneUpdated = True
            
            if not dynamicOrangeZoneUpdated:
                newDynamicOrangeZones.append({"district": district, "startDay": day, "endDay": day + modelParameter["dzOZOffsetDays"]});
        
        ## Red zone
        if districtISum*modelParameter["aR"] > modelParameter["dzRZThreshold"]:
            #print(district, ' in Red Zone')
            dynamicRedZoneUpdated = False;
            for dynamicRedZone in dynamicRedZones:
                if district == dynamicRedZone["district"]:
                    dynamicRedZone["endDay"] =  max(dynamicRedZone["endDay"], day+modelParameter["dzRZOffsetDays"])
                    dynamicRedZoneUpdated = True
            
            if not dynamicRedZoneUpdated:
                newDynamicRedZones.append({"district": district, "startDay": day, "endDay": day + modelParameter["dzRZOffsetDays"]});
                

        

    dynamicZones["dynamicRedZones"]     = dynamicZones["dynamicRedZones"]     +   newDynamicRedZones
    dynamicZones["dynamicOrangeZones"]  = dynamicZones["dynamicOrangeZones"]  +  newDynamicRedZones
    
    return dynamicZones;

In [160]:
def getDynamicOrangeZoneMitigationMatrix(day, dynamicOrangeZones, modelParameter):
    M = np.ones((r,r))
    for dynamicOrangeZone in dynamicOrangeZones:
        if day >= dynamicOrangeZone['startDay'] and day <= dynamicOrangeZone['endDay']:
            district = dynamicOrangeZone['district']
            regionsInDistrictList = initDataDF.index[initDataDF['district'] == district]
            for regionsInDistrictIndex in regionsInDistrictList:
                fillRegion(M, regionsInDistrictIndex, modelParameter["eOZ"])
                
            
    return M

In [161]:
def getDynamicRedZoneMitigationMatrix(day, dynamicRedZones, modelParameter):
    M = np.ones((r,r))
    for dynamicRedZone in dynamicRedZones:
        if day >= dynamicRedZone['startDay'] and day <= dynamicRedZone['endDay']:
            district = dynamicRedZone['district']
            regionsInDistrictList = initDataDF.index[initDataDF['district'] == district]
            for regionsInDistrictIndex in regionsInDistrictList:
                fillRegion(M, regionsInDistrictIndex, modelParameter["eRZ"])
                
            
    return M


## Function definitions

In [162]:
def getMitigationMatrix(day, dynamicHotSpots, dynamicZones, userParameterData, r, initDataDF, modelParameter):    
    ## Effective mitigation matrix
    Meffective = Mn
    
    if isMitigationEnabled(day,"completeLockDowns", userParameterData):
        Meffective = np.minimum(Meffective, Mld)
    
    if isMitigationEnabled(day,"districtLockDowns", userParameterData):
        Meffective = np.minimum(Meffective, Mdb)
    
    if isMitigationEnabled(day,"hotSpots", userParameterData):
        Mhs = getHotSpotMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter)
        Meffective = np.minimum(Meffective, Mhs)
    
    if isMitigationEnabled(day,"redZones", userParameterData):
        Mrz = getRedZoneMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter)
        Meffective = np.minimum(Meffective, Mrz)
    
    if isMitigationEnabled(day,"orangeZones", userParameterData):
        Moz = getOrangeZoneMitigationMatrix(day, r, userParamterData, initDataDF, modelParameter)
        Meffective = np.minimum(Meffective, Moz)
    
    if isMitigationEnabled(day,"dynamicHotSpots", userParameterData):
        Mdhs = getDynamicHotSpotMitigationMatrix(day, dynamicHotSpots, modelParameter)
        Meffective = np.minimum(Meffective, Mdhs)
    
    if isMitigationEnabled(day,"dynamicZones", userParameterData):
        if 'dynamicOrangeZones' in dynamicZones:
            MdzOZ = getDynamicOrangeZoneMitigationMatrix(day, dynamicZones['dynamicOrangeZones'], modelParameter)
            Meffective = np.minimum(Meffective, MdzOZ)
        if 'dynamicRedZones' in dynamicZones:
            MdzRZ = getDynamicRedZoneMitigationMatrix(day, dynamicZones['dynamicRedZones'], modelParameter)
            Meffective = np.minimum(Meffective, MdzRZ)

    if isMitigationEnabled(day,"breakTheChains", userParameterData):
        Meffective = np.multiply(Meffective, Mbc)
          
    return Meffective


In [163]:
def computeContactMatrix(day, dynamicHotSpots, dynamicZones, userParameterData, r, initDataDF, modelParameter):    
    Meffective = getMitigationMatrix(day, dynamicHotSpots, dynamicZones, userParameterData, r, initDataDF, modelParameter)        
    Cmitigated = np.multiply(Meffective,Wn)    
    np.fill_diagonal(Cmitigated, Cmitigated.diagonal() + modelParameter["ch"])
    return Cmitigated
    


## Run the model for all regions

### Data structures for SEIR model

### Initialisation (Day 0)

In [164]:
def runloop_data_initialisation(initDataDF):
    S = np.zeros((NO_OF_DAYS,r))
    E = np.zeros((NO_OF_DAYS,r))
    I = np.zeros((NO_OF_DAYS,r))
    R = np.zeros((NO_OF_DAYS,r))

    E[0] = initDataDF['E']
    I[0] = initDataDF['I']
    R[0] = initDataDF['R']
    S[0] = popDataDF['N'] - E[0] - I[0] - R[0]
    
    dynamicHotSpots = []
    dynamicZones = {}
    
    return S,E,I,R, dynamicHotSpots, dynamicZones

### Time loop

In [165]:
def timeloop(initDataDF, userParameterData, modelParameter, r, popDataDF):

    S,E,I,R, dynamicHotSpots, dynamicZones = runloop_data_initialisation(initDataDF)

    for day in range(1,NO_OF_DAYS):
        if isMitigationEnabled(day,"dynamicHotSpots", userParameterData):
            dynamicHotSpots = updateDynamicHotSpots(day, I[day-1], dynamicHotSpots, modelParameter);
    
        if isMitigationEnabled(day,"dynamicZones", userParameterData):
            dynamicZones = updateDynamicZones(day, I[day-1], districts, dynamicZones, initDataDF, modelParameter);
        
        C = computeContactMatrix(day, dynamicHotSpots, dynamicZones, userParameterData, r, initDataDF, modelParameter)
    
        IbyN = I[day-1] / popDataDF['N']
        leaveS = modelParameter["pi"] * S[day-1] * np.matmul(C, IbyN)
        leaveE = E[day-1]/modelParameter["tE"]
        leaveI = I[day-1]/modelParameter["tI"]
    
        S[day] = S[day-1] - leaveS
        E[day] = E[day-1] + leaveS - leaveE
        I[day] = I[day-1] + leaveE - leaveI
        R[day] = R[day-1] + leaveI
        
        return dynamicHotSpots, dynamicZones, I

In [166]:
dynamicHotSpots, dynamicZones, I = timeloop(initDataDF, userParameterData, modelParameter, r, popDataDF)

In [167]:
if isMitigationEnabled(1,"dynamicHotSpots", userParameterData):
    hotSpotsDF = pd.DataFrame(dynamicHotSpots)
    hotSpotIdx = list(hotSpotsDF['regionIndex'])
    hotSpotsDF['duration'] =  hotSpotsDF['endDay'] - hotSpotsDF['startDay']
    hotSpotsDF['name'] = list(popDataDF['name'][hotSpotIdx])
    hotSpotsDF.sort_values(by = 'name', inplace=True)
    totalHotSpotDays = hotSpotsDF['duration'].sum()
    print("totalHotSpotDays =", totalHotSpotDays)

if isMitigationEnabled(1,"dynamicZones", userParameterData):
    redZonesDF = pd.DataFrame(dynamicZones['dynamicRedZones'])
    orangeZonesDF = pd.DataFrame(dynamicZones['dynamicOrangeZones'])
    redZonesDF.sort_values(by = 'district', inplace=True)
    orangeZonesDF.sort_values(by = 'district', inplace=True)
    
redZonesDF


KeyError: 'district'

### Store I to a dataframe

In [112]:
resultsIDF = pd.DataFrame(I, columns=popDataDF['name'])
resultsIDF['Palakkad(M)__Palakkad']
resultsIDF
# resultsIDF.to_csv("../output/" + district + "_results_v2.csv")

name,ARNagar__Malappuram,Adat__Thrissur,Adichanalloor__Kollam,Adimali__Idukki,Adoor(M)__Pathanamthitta,Agali__Palakkad,Aiykkaranadu__Ernakulam,Ajanur__Kasaragod,Akalakunnam__Kottayam,Akathethara__Palakkad,...,Vilappil__Thiruvananthapuram,Vilavoorkal__Thiruvananthapuram,Vilayoor__Palakkad,Villiyappally__Kozhikode,Vithura__Thiruvananthapuram,Vorkady__Kasaragod,Vythiri__Wayanad,Wandoor__Malappuram,West Eleri__Kasaragod,West Kallada__Kollam
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000012,0.000087,0.000007,0.000059,0.000015,0.000351,0.000018,0.000011,0.000010,0.010520,...,0.000005,0.000004,0.000119,0.000017,0.000004,0.000009,0.000005,0.000034,0.000010,0.000005
3,0.000027,0.000203,0.000017,0.000138,0.000035,0.000819,0.000043,0.000026,0.000024,0.024547,...,0.000011,0.000008,0.000277,0.000041,0.000008,0.000020,0.000012,0.000079,0.000024,0.000011
4,0.000045,0.000334,0.000028,0.000227,0.000058,0.001345,0.000071,0.000043,0.000039,0.040227,...,0.000019,0.000014,0.000456,0.000067,0.000014,0.000033,0.000020,0.000129,0.000040,0.000019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,0.518113,1.405094,1.496241,3.857461,1.866532,0.841983,0.821319,4.465260,1.130614,2.820117,...,1.348263,1.179404,0.997202,7.987690,0.865879,2.474810,0.943947,0.910549,3.183104,0.886335
361,0.518205,1.405610,1.496434,3.853602,1.864680,0.844048,0.821884,4.465755,1.130476,2.815329,...,1.348529,1.179659,0.998892,7.996874,0.866173,2.474048,0.941666,0.910002,3.182613,0.886307
362,0.518294,1.406123,1.496625,3.849741,1.862833,0.846108,0.822446,4.466232,1.130335,2.810539,...,1.348792,1.179911,1.000571,8.005798,0.866465,2.473279,0.939388,0.909452,3.182108,0.886278
363,0.518381,1.406633,1.496812,3.845877,1.860993,0.848161,0.823006,4.466691,1.130191,2.805747,...,1.349054,1.180161,1.002239,8.014467,0.866757,2.472503,0.937112,0.908901,3.181589,0.886248


### Filter out a selected list of LSGs

In [113]:
statsDF = pd.DataFrame(resultsIDF.max(), columns=['peakVal'])
statsDF['peakDay'] = resultsIDF.idxmax()
statsDF.sort_values(by=['peakVal'], inplace=True)

statsDF['500+Day'] = resultsIDF.clip(0,500).idxmax()

statsDF.head(20)

idxHead = list(np.arange(0,r) < 3)
idxPeakC = statsDF.index.str.contains('\(C\)',regex=True)
idxPeakM = statsDF.index.str.contains('\(M\)',regex=True)
idxPeakM = idxPeakM & list(np.arange(0,r) < 20)
idxTail = list(np.arange(0,r) >= r-25)

idxSelected = idxHead | idxPeakC | idxPeakM | idxTail

statsSelectedDF = statsDF[idxSelected]
statsSelectedDF

,peakVal,peakDay,500+Day
name,,,
Nelliyampathi__Palakkad,0.124589,364,364
Keezhuparambu__Malappuram,0.323072,364,364
Muthuvalloor__Malappuram,0.346767,364,364
Onchiyam__Kozhikode,14.124610,333,333
Edacheri__Kozhikode,14.434507,333,333
Thrissur(C)__Thrissur,14.539619,364,364
Kunnothuparambu__Kannur,14.903440,363,363
Dharmadom__Kannur,15.425863,348,348
Eramala__Kozhikode,16.040501,337,337


In [114]:
resultsSelectedDF = resultsIDF.filter(statsSelectedDF.index)
figSelected = resultsSelectedDF.iplot(asFigure=True, xTitle="Days",
                    yTitle="I", title="Selected LSGs")
figSelected.show()

### District-wise plots

In [124]:
districts
#resultsIDF
resultsIDF.filter(like="Kozhikode", axis=1).sum(axis=1)

0        0.000000
1        0.000000
2        0.002543
3        0.005934
4        0.009763
          ...    
360    398.029160
361    398.109514
362    398.182019
363    398.246835
364    398.304113
Length: 365, dtype: float64

In [125]:
resultsDistDF = pd.DataFrame()
for dist in districts:
    print(dist)
    resultsDistDF[dist] = resultsIDF.filter(like=dist, axis=1).sum(axis=1)
resultsDistDF

Malappuram
Thrissur
Kollam
Idukki
Pathanamthitta
Palakkad
Ernakulam
Kasaragod
Kottayam
Alappuzha
Kannur
Wayanad
Thiruvananthapuram
Kozhikode
None


TypeError: Must pass either `items`, `like`, or `regex`

In [117]:
figDist = resultsDistDF.iplot(asFigure=True, xTitle="Days",
                    yTitle="I", title="Districts")
figDist.show()

In [118]:
districtNames

['Malappuram',
 'Thrissur',
 'Kollam',
 'Idukki',
 'Pathanamthitta',
 'Palakkad',
 'Ernakulam',
 'Kasaragod',
 'Kottayam',
 'Alappuzha',
 'Kannur',
 'Wayanad',
 'Thiruvananthapuram']